In [5]:
import numpy as np
import math
import pandas as pd
from sklearn import datasets
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression 

In [6]:
def h(a):
    q = math.exp(-a)
    return 1/(1+q)

In [7]:
def step_gradient(data,lr,m):
    N = len(data)
    n = len(m)
    m_slope = np.zeros(n)
    m_new = np.zeros(n)
    for i in range(N):
        a = np.dot(m,data[i][:-1])
        for j in range(n):
            x = data[i,j]
            y = data[i,n]
            sig = h(a)
            m_slope[j] += (1/N)*(sig-y)*x
            m_new[j] = m[j]-lr*m_slope[j]
    return m_new


In [8]:
def cost(data,m):
    total_cost = 0
    N = len(data)
    n = len(m)
    for i in range(N):
        a = np.dot(m,data[i][:-1])
        sig = h(a)
#         print(sig)
        y = data[i,n]
        t1 = y*math.log(sig)
        t2 = (1-y)*math.log(1-sig)
        total_cost += (-1/N)*(t1+t2)
    return total_cost

In [9]:
def gd(data,lr,ni):
    m=np.zeros(len(data[0][:-1]))
    for i in range(ni):
        m = step_gradient(data,lr,m)
        print("Cost ",i, " ", cost(data,m))
    return m

In [10]:
def predict(m, test_data):
    N = len(test_data)
    c = m[-1]
    m = m[:-1]
    li = []
    for i in range (N):
        a = np.dot(m,test_data[i])
        a += c
        sig = h(a)
        li.append(sig)
    ans=[]
    for x in li:
        if x>=0.65:
            ans.append(1)
        else:
            ans.append(0)
    return ans

In [17]:
def run():
    data = pd.read_csv("C:/Users/Dhruv/Desktop/Machine Learning/Datasets/Titanic/Project/0000000000002429_training_titanic_x_y_train.csv", delimiter=",")
    data_test = pd.read_csv("C:/Users/Dhruv/Desktop/Machine Learning/Datasets/Titanic/Project/0000000000002429_test_titanic_x_test.csv", delimiter=",")
    # cancer = datasets.load_breast_cancer()
    # df = pd.DataFrame(data)
    # df.columns = cancer.feature_names
    # df['Y'] = cancer.target
    # X = df.values
    del data['Name']
    del data['Ticket']
    del data['Cabin']
    def f(s):
        if s=="male":
            return 0
        else:
            return 1
    def f2(s):
        if s=="S":
            return 1
        elif s=='C':
            return 2
        else:
            return 3
    data["Gender"] = data["Sex"].apply(f)
    data['Embark'] = data['Embarked'].apply(f2)
    del data['Sex']
    del data['Embarked']

    del data_test['Name']
    del data_test['Ticket']
    del data_test['Cabin']
    def f(s):
        if s=="male":
            return 0
        else:
            return 1
    def f2(s):
        if s=="S":
            return 1
        elif s=='C':
            return 2
        else:
            return 3
    data_test["Gender"] = data_test["Sex"].apply(f)
    data_test['Embark'] = data_test['Embarked'].apply(f2)
    del data_test['Sex']
    del data_test['Embarked']
    data_test.Age.fillna(data_test.Age.mean(),inplace=True)


    sur=data[data.Survived==1]
    mean1 = sur.Age.mean()
    sur
    nosur=data[data.Survived==0]
    mean0 = nosur.Age.mean()
    print(mean0)
    notsurvived = nosur.copy()
    notsurvived.Age.fillna(mean0,inplace=True)
    notsurvived
    print(mean1)
    survived = sur.copy()
    survived.Age.fillna(mean1,inplace=True)
    survived
    # sur = df[df.Survived==0]
    survived = survived.append(notsurvived)
    def f3(s):
        return s
    survived['Y'] = survived['Survived'].apply(f3)
    del survived['Survived']
    survived
    X = survived.values
    
    scalar = preprocessing.StandardScaler()
    input = X[:,:-1]
    Y = X[:,-1]
    
    square = pd.DataFrame(input)
    square.columns = ['Pclass','Age','SibSp','Parch','Fare','Gender','Embark']
    square['Age'] *= 2
    
    for i in  range(7):
        for j in range(i,7):
            first = square.columns[i]
            second = square.columns[j]
            name =  first + "_" + second
            square[name] = square[first]*square[second]
    
    input = square.values
    scalar.fit(input)
    input = scalar.transform(input)
    Y = Y.tolist()
    input = input.tolist()
    for i in range(len(input)):
        input[i].append(Y[i])
    input = np.array(input)
    Y = np.array(Y)
    
#     data = np.insert(input,-1,1,axis=1)
    lr = 0.5
    ni = 1000
    clf = LogisticRegression()
    X_train = input[:,:-1]
    Y_train = input[:,-1]
    clf.fit(X_train,Y_train)
    
#     m = gd(data,lr,ni)
    
    data_test['Age'] *=2
    for i in  range(7):
        for j in range(i,7):
            first = data_test.columns[i]
            second = data_test.columns[j]
            name =  first + "_" + second
            data_test[name] = data_test[first]*data_test[second]
    
    
    test = data_test.values
    test = scalar.transform(test)
    Y_pred_test = clf.predict(test)
#     Y_pred_test = predict(m,test)
    print(Y_pred_test)
    np.savetxt("C:/Users/Dhruv/Desktop/Machine Learning/Datasets/Titanic/Project/predictions_inbuilt.csv",Y_pred_test)

#     print(m)

In [18]:
run()

30.857142857142858
28.138157894736842
[1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1.
 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1.
 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0.
 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1.
 0. 0. 0. 0. 1. 0. 0.]


In [ ]:
[0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0]